In [48]:
def bls_pull(registration_key, area_code, start_year, end_year):
    
    import requests
    import json
    import pandas as pd



    industries = {'11': 'natural_resources_mining',
                  '12': 'construction',
                  '13': 'manufacturing',
                  '21': 'trade_transportation_utilities',
                  '22': 'information',
                  '23': 'financial_activities',
                  '24': 'professional_business_services',
                  '25': 'education_health_services',
                  '26': 'leisure_hospitality',
                  '27': 'other_services',
                  '28': 'public_administration',
                  '29': 'unclassified'}

    data_names = {'1': 'all_employees',
                  '4': 'avg_weekly_pay'}




    json_dicts = {}

    for i_code, i_name in industries.items():

        for d_code, d_name in data_names.items():

            headers = {'Content-type': 'application/json'}
            data = json.dumps({"seriesid": ['ENU'+ area_code + d_code + '0510' + i_code],'registrationkey': registration_key, 
                               "startyear": start_year, "endyear": end_year})
            p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
            json_data = json.loads(p.text)

            json_dicts[i_name + '_' + d_name] = json_data['Results']['series'][0]['data']

    bad_keys = []
    for i in json_dicts:
        if len(json_dicts[i]) == 0:
            bad_keys.append(i) 

    for i in bad_keys:
        del json_dicts[i]




    df_dicts = {}

    for i in json_dicts:
        df_dicts[i] = pd.DataFrame(json_dicts[i])

    for i in df_dicts:

        if 'avg_weekly_pay' in i:
            df_dicts[i][i] = df_dicts[i]['value']
            df_dicts[i]['date'] = df_dicts[i].apply(lambda x: x['year'] + x['period'][1:], axis=1)
            df_dicts[i] = pd.concat([df_dicts[i]['date'], df_dicts[i][i]], axis=1)

        elif 'all_employees' in i:
            df_dicts[i][i] = df_dicts[i]['value']
            df_dicts[i]['date'] = df_dicts[i].apply(lambda x: x['year'] + x['period'][1:], axis=1)
            df_dicts[i] = pd.concat([df_dicts[i]['date'], df_dicts[i][i]], axis=1)

        cols = [j for j in df_dicts[i] if j != 'date']
        df_dicts[i][cols] = df_dicts[i][cols].apply(pd.to_numeric, errors='coerce')

    for df_key in df_dicts:

        wp_mask = df_key.split('all')[0] + 'avg_weekly_pay'

        if 'all_employees' in df_key:

            df_dicts[df_key][wp_mask] = 0

            for index, period in df_dicts[df_key]['date'].iteritems():
                if period[4:] in ['01', '02', '03']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'01')][wp_mask].iloc[0]
                if period[4:] in ['04', '05', '06']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'02')][wp_mask].iloc[0]
                if period[4:] in ['07', '08', '09']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'03')][wp_mask].iloc[0]
                if period[4:] in ['10', '11', '12']:
                    df_dicts[df_key].loc[index, wp_mask] = 4 * df_dicts[wp_mask][df_dicts[wp_mask]['date']==(period[:4]+'04')][wp_mask].iloc[0]

            df_dicts[df_key].rename(columns={wp_mask: df_key.split('all')[0] + 'avg_monthly_pay'}, inplace=True)
            df_dicts[df_key][df_key.split('all')[0] + 'aggr_value'] = df_dicts[df_key][df_key] * df_dicts[df_key][df_key.split('all')[0] + 'avg_monthly_pay']

    merged_df = pd.DataFrame(columns=['date'])

    for df_key in df_dicts:
        if 'all_employees' in df_key:
            merged_df = pd.merge(merged_df, df_dicts[df_key], how = 'outer', on ='date')

    temp_merge = merged_df.drop('date', axis=1)
    temp_cols = [i for i in temp_merge.columns if 'aggr_val' in i]
    merged_df['total_industries_aggr_value'] = temp_merge[temp_cols].apply(lambda x: sum(x), axis=1)
    
    return merged_df


In [ ]:
#EW API 9ff92c5c737a495297232e59bb594f55
#AIDIAN API 2738330500f74641a2f99b3ecd9831e2

In [58]:
#Livingston Louisana 2016 August Flood
livingston = bls_pull(registration_key='2738330500f74641a2f99b3ecd9831e2', start_year='2014', end_year='2017', area_code='22063') 
livingston['date'] = [x[:4]+'-'+x[4:8] for x in livingston['date']]

In [80]:
import pygal 

l = pygal.Line()
l.title =" "
l.x_labels = [x for x in livingston['date'][::-1]] 
for x in livingston.columns[1:-1:3]:
    l.add(str(x), [x for x in livingston[x]])
l.render_to_file(filename= 'Livingston2014-2017.html')

In [65]:
livingston.to_csv('livingston2014-2017.csv')

In [62]:
# Greenbreier West Virginia 2016 Flood June 23–24, 2016
greenbrier = bls_pull(registration_key='2738330500f74641a2f99b3ecd9831e2', start_year='2014', end_year='2017', area_code='54025')
greenbrier['date'] = [x[:4]+'-'+x[4:8] for x in greenbrier['date']]

In [81]:
import pygal 

l = pygal.Line()
l.title =" "
l.x_labels = [x for x in greenbrier['date'][::-1]] 
for x in greenbrier.columns[1:-1:3]:
    l.add(str(x), [x for x in greenbrier[x]])
l.render_to_file(filename= 'Greenbrier2014-2017.html')

In [67]:
greenbrier.to_csv('greenbrier2014-2017.csv')

In [154]:
# Charleston South Carolina 2015 North American Storm Complex September 29, 2015 to October 7, 2015
charleston = bls_pull(registration_key='2738330500f74641a2f99b3ecd9831e2', start_year='2014', end_year='2017', area_code='45019')
charleston['date'] = [x[:4]+'-'+x[4:8] for x in charleston['date']]

In [157]:
import pygal 

l = pygal.Line()
l.title =" "
l.x_labels = [x for x in charleston['date'][::-1]] 
for x in charleston.columns[1:-1:3]:
    l.add(str(x), [x for x in charleston[x]])
l.render_to_file(filename= 'Charleston2014-2017.html')

In [158]:
charleston.to_csv('charleston2014-2017.csv')

In [134]:
# Okanogan Washington Complex Fires August 15, 2015 – September 19, 2015
okanogan = bls_pull(registration_key='2738330500f74641a2f99b3ecd9831e2', start_year='2014', end_year='2017', area_code='53047')
okanogan['date'] = [x[:4]+'-'+x[4:8] for x in okanogan['date']]

In [135]:
import pygal 

l = pygal.Line()
l.title =" "
l.x_labels = [x for x in okanogan['date'][::-1]] 
for x in okanogan.columns[1:-1:3]:
    l.add(str(x), [x for x in okanogan[x]])
l.render_to_file(filename= 'okanogan2014-2017.html')

In [137]:
okanogan.to_csv('okanogan2014-2017.csv')

In [ ]:
# Limestone Alabama torando 01083 april 2011

In [146]:
# Limestone Alabama torando 01083 april 2011
limestone = bls_pull(registration_key='2738330500f74641a2f99b3ecd9831e2', start_year='2010', end_year='2013', area_code='01083')
limestone['date'] = [x[:4]+'-'+x[4:8] for x in limestone['date']]

In [147]:
import pygal 

l = pygal.Line()
l.title =" "
l.x_labels = [x for x in limestone['date'][::-1]] 
for x in limestone.columns[1:-1:3]:
    l.add(str(x), [x for x in limestone[x]])
l.render_to_file(filename= 'limestone2010-2013.html')

In [148]:
limestone.to_csv('limestone2010-2013.csv')

In [149]:
# Hoboken NJ, 34017, Hurricane Sandy 2009-2013
Hoboken = bls_pull(registration_key='2738330500f74641a2f99b3ecd9831e2', start_year='2009', end_year='2013', area_code='34017')
Hoboken['date'] = [x[:4]+'-'+x[4:8] for x in Hoboken['date']]

In [150]:
import pygal 

l = pygal.Line()
l.title =" "
l.x_labels = [x for x in Hoboken['date'][::-1]] 
for x in Hoboken.columns[1:-1:3]:
    l.add(str(x), [x for x in Hoboken[x]])
l.render_to_file(filename= 'Hoboken2009-2013.html')

In [151]:
Hoboken.to_csv('Hoboken2009-2013.csv')

In [159]:
# Jefferson Texas, 34017, Hurricane Harvey 2014-2018 (occured Sept -August 2017)
Jefferson = bls_pull(registration_key='2738330500f74641a2f99b3ecd9831e2', start_year='2014', end_year='2018', area_code='48245')
Jefferson['date'] = [x[:4]+'-'+x[4:8] for x in Jefferson['date']]

In [160]:
import pygal 

l = pygal.Line()
l.title =" "
l.x_labels = [x for x in Jefferson['date'][::-1]] 
for x in Jefferson.columns[1:-1:3]:
    l.add(str(x), [x for x in Jefferson[x]])
l.render_to_file(filename= 'Jefferson2014-2018.html')

In [161]:
Jefferson.to_csv('Jefferson2014-2018.csv')

In [ ]:
# 29145 Newton County, Mo - may 2011
Newton = bls_pull(registration_key='2738330500f74641a2f99b3ecd9831e2', start_year='2009', end_year='2013', area_code='29145')
Newton['date'] = [x[:4]+'-'+x[4:8] for x in Newton['date']]

In [ ]:
import pygal 

l = pygal.Line()
l.title =" "
l.x_labels = [x for x in Newton['date'][::-1]] 
for x in Newton.columns[1:-1:3]:
    l.add(str(x), [x for x in Newton[x]])
l.render_to_file(filename= 'Newton2009-2013.html')

In [ ]:
Newton.to_csv('Newton2009-2013.csv')

In [116]:
df = pd.read_csv('C:\\Users\\ericp\\OneDrive\\Documents\\GitHub\\project-4\\hurricane_katrina_new_orleans.csv')

In [120]:
df['date'] = df['date'].astype(str)

In [128]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [129]:
df['date'] = [x[:4]+'-'+x[4:8] for x in df['date']]

In [133]:
import pygal 

l = pygal.Line()
l.title =" "
l.x_labels = [x for x in df['date'][::-1]] 
for x in df.columns[1:-1:3]:
    l.add(str(x), [x for x in df[x]])
l.render_to_file(filename= 'ParishOrleansKatrina2014-2017.html')

In [132]:
df

,date,natural_resources_mining_all_employees,natural_resources_mining_avg_monthly_pay,natural_resources_mining_aggr_value,construction_all_employees,construction_avg_monthly_pay,construction_aggr_value,manufacturing_all_employees,manufacturing_avg_monthly_pay,manufacturing_aggr_value,...,leisure_hospitality_all_employees,leisure_hospitality_avg_monthly_pay,leisure_hospitality_aggr_value,other_services_all_employees,other_services_avg_monthly_pay,other_services_aggr_value,unclassified_all_employees,unclassified_avg_monthly_pay,unclassified_aggr_value,total_industries_aggr_value
0,2006--12,3709,9088,33707392,5759,4220,24302980,7345,4316,31701020,...,28816,1824,52560384,4943,2448,12100464,549,2760,1515240,499144676
1,2006--11,3718,9088,33789184,5883,4220,24826260,7223,4316,31174468,...,27555,1824,50260320,4867,2448,11914416,511,2760,1410360,493510160
2,2006--10,3745,9088,34034560,5993,4220,25290460,7126,4316,30755816,...,26929,1824,49118496,4931,2448,12071088,517,2760,1426920,488854208
3,2006--09,3816,8300,31672800,6131,4108,25186148,7009,4092,28680828,...,25135,1764,44338140,4351,2164,9415564,427,2856,1219512,423121208
4,2006--08,3799,8300,31531700,6216,4108,25535328,6917,4092,28304364,...,24586,1764,43369704,4243,2164,9181852,388,2856,1108128,421259416
5,2006--07,3779,8300,31365700,6295,4108,25859860,6903,4092,28247076,...,24104,1764,42519456,4141,2164,8961124,360,2856,1028160,416013508
6,2006--06,3785,8388,31748580,7634,4020,30688680,6749,4172,28156828,...,23684,1856,43957504,4103,2228,9141484,360,3244,1167840,436262996
7,2006--05,3681,8388,30876228,7458,4020,29981160,6604,4172,27551888,...,22433,1856,41635648,4052,2228,9027856,328,3244,1064032,425293856
8,2006--04,3697,8388,31010436,7309,4020,29382180,6522,4172,27209784,...,21770,1856,40405120,3893,2228,8673604,304,3244,986176,420822476
9,2006--03,3758,11760,44194080,7341,3940,28923540,6517,4332,28231644,...,21945,1872,41081040,3732,2304,8598528,268,5220,1398960,462705468
